In [1]:
import numpy as np
import pandas as pd
import math
from scipy.stats import norm
import matplotlib.pyplot as plt


def d1(S, K, T, r, q, sigma):
    """Calculates d1 (BSM)."""
    return (math.log(S / K) + (r - q + 0.5 * sigma**2) * T) / (sigma * math.sqrt(T))

def d2(S, K, T, r, q, sigma, d1_val=None):
    """Calculates d2 (BSM)."""
    if d1_val is None:
        d1_val = d1(S, K, T, r, q, sigma)
    return d1_val - sigma * math.sqrt(T)

def bs_call(S, K, T, r, q, sigma):
    """Calculate the value for a European call option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return S * math.exp(-q * T) * norm.cdf(d1_val) - K * math.exp(-r * T) * norm.cdf(d2_val)

def bs_put(S, K, T, r, q, sigma):
    """Calculate the value for a European put option (BSM)."""
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma, d1_val)
    return K * math.exp(-r * T) * norm.cdf(-d2_val) - S * math.exp(-q * T) * norm.cdf(-d1_val)

from enum import IntEnum

class CallPutType(IntEnum):
    '''define a class of CallPutType'''
    Call = 0
    Put = 1
class EuroAmerType(IntEnum):
    EuropeanOption = 0
    AmericanOption = 1
    
def delta_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    if call_put_type == CallPutType.Call:
        delta = np.exp(-q*T) * norm.cdf(d1_val)
    elif call_put_type == CallPutType.Put:
        delta = -np.exp(-q*T) * norm.cdf(-d1_val)
    else:
        raise ValueError("invalid call_put_type")
    return delta

def gamma_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    # gammas of C and P are the same for european options
    gamma = np.exp(-q*T) / (S*sigma*np.sqrt(T)) * (1/np.sqrt(2*np.pi)) * np.exp(-d1_val**2/2)
    return gamma

def vega_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    # vegas of C and P are the same for european options
    vega = S * np.exp(-q*T) * np.sqrt(T) * (1/np.sqrt(2*np.pi)) * np.exp(-d1_val**2/2)
    return vega 

def theta_bs_european(S, K, T, r, q, sigma, call_put_type):
    d1_val = d1(S, K, T, r, q, sigma)
    d2_val = d2(S, K, T, r, q, sigma)
    if call_put_type == CallPutType.Call:
        theta = (
            -(S*sigma*np.exp(-q*T)) / (2*np.sqrt(2*np.pi*T)) * np.exp(-d1_val**2/2)
            + q * S * np.exp(-q*T) * norm.cdf(d1_val)
            - r * K * np.exp(-r*T) * norm.cdf(d2_val)
        )
    elif call_put_type == CallPutType.Put:
        theta = (
            -(S*sigma*np.exp(-q*T)) / (2*np.sqrt(2*np.pi*T)) * np.exp(-d1_val**2/2) 
            - q * S * np.exp(-q*T) * norm.cdf(-d1_val) 
            + r * K * np.exp(-r*T) * norm.cdf(-d2_val)
        )
    else:
        raise ValueError("Invalid call_put_type")
    return theta

In [2]:
## Option Parameters. ##
S0 = 54
K = 50
T = 1
r = 0.0375
q = 0.01
sigma = 0.29


V_BS = bs_put(S0, K, T, r, q, sigma)
Delta_BS = delta_bs_european(S0, K, T, r, q, sigma, CallPutType.Put) 
Gamma_BS = gamma_bs_european(S0, K, T, r, q, sigma, CallPutType.Put)
Theta_BS = theta_bs_european(S0, K, T, r, q, sigma, CallPutType.Put)

black_scholes_res = pd.DataFrame({
    "V_BS": V_BS,
    "Delta_BS": Delta_BS,
    "Gamma_BS": Gamma_BS,
    "Theta_BS": Theta_BS,
}, index=[0])
print(black_scholes_res)

       V_BS  Delta_BS  Gamma_BS  Theta_BS
0  3.579428 -0.303654    0.0222  -2.13695


## Binomial Tree Methods for European Options

In [3]:
def delta_1(V_1_0, V_1_1, S_1_0, S_1_1):
    return (V_1_0 - V_1_1) / (S_1_0 - S_1_1)

def gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2):
    upper_1 = (V_2_0 - V_2_1) / (S_2_0 - S_2_1)
    upper_2 = (V_2_1 - V_2_2) / (S_2_1 - S_2_2)
    lower = (S_2_0 - S_2_2) / 2
    return (upper_1 - upper_2) / lower

def theta_1(V_2_1, V_0_0, T, N):
    dt = T/N
    return (V_2_1 - V_0_0) / (2*dt)

In [4]:
def u_(T, sigma, N):
    """S1 = S0 * u_"""
    dt = T/N
    return np.exp(sigma*np.sqrt(dt))

def d_(T, sigma, N):
    """S1 = S0 * d_"""
    return 1 / u_(T, sigma, N)

def payoffs(V, i, S_temp, K, flag1, flag2):
    if not flag1 and not flag2: # European call
        V[i] = max(S_temp - K, 0)
    elif flag1 and flag2: # American put
        V[i] = max(K - S_temp, 0)
    elif flag1: # European put
        V[i] = max(K - S_temp, 0)
    else: # American call
        V[i] = max(S_temp - K, 0)


In [5]:
def approximation_error(reference, target):
    return abs(reference-target)

def linear_approximation_error(reference, target, N):
    return N * approximation_error(reference, target)

def quadratic_approximation_error(reference, target, N):
    return N**2 * approximation_error(reference, target)

In [6]:
## output module of Binomial Tree Error Approximate
def generate_geometric_sequence(start, ratio, length):
    """
    e.g. start = 10, ratio = 2, length = 5
    return [10, 20, 40, 80, 160]
    """
    progression = np.empty(length, dtype=int)
    for i in range(0, length):
        curr_term = start * pow(ratio, i)
        progression[i] = int(curr_term)
    return progression
    
def simulation(S0, K, T, r, q, sigma, N, CallPut, EuroAmer, method_name,V_exact,Delta_exact,Gamma_exact,Theta_exact):
        # preparation
        f1 = eval(f'{method_name}_pricer_early_stop')
        f2 = eval(f'{method_name}_pricer')
        V_1_0, V_1_1 = f1(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer,
            early_stop=1
        )
        V_2_0, V_2_1, V_2_2 = f1(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer,
            early_stop=2
        )
        V_0_0 = f2(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer
        )
        u = u_(T, sigma, N)
        d = d_(T, sigma, N)
        S_1_0 = S0 * u
        S_1_1 = S0 * d
        S_2_0 = S0 * u**2
        S_2_1 = S0 * u * d
        S_2_2 = S0 * d**2
        
        # value from binomial tree pricer
        EP_ABT = f2(S0, K, T, r, q, sigma, N, CallPut, EuroAmer)
        # approximation error
        EP_ABT_approximation_error = approximation_error(EP_ABT, V_exact)
        EP_ABT_linear_approximation_error = linear_approximation_error(EP_ABT, V_exact, N)
        EP_ABT_quadratic_approximation_error = quadratic_approximation_error(EP_ABT, V_exact, N)
        
        EP_ABT_delta = delta_1(V_1_0, V_1_1, S_1_0, S_1_1)
        EP_ABT_delta_error = approximation_error(EP_ABT_delta , Delta_exact)
        EP_ABT_gamma = gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2)
        EP_ABT_gamma_error = approximation_error(EP_ABT_gamma, Gamma_exact)
        EP_ABT_theta = theta_1(V_2_1, V_0_0, T, N)
        EP_ABT_theta_error = approximation_error(EP_ABT_theta, Theta_exact)

        return EP_ABT,EP_ABT_approximation_error,EP_ABT_linear_approximation_error, EP_ABT_quadratic_approximation_error, EP_ABT_delta, EP_ABT_delta_error, EP_ABT_gamma, EP_ABT_gamma_error,EP_ABT_theta,EP_ABT_theta_error

        
def approximate_binomial_tree_res(S0, K, T, r, q, sigma, CallPut, EuroAmer, method_name,V_exact,Delta_exact,Gamma_exact,Theta_exact):
    # Average Binomial Tree
    # number of steps to use 
    N = generate_geometric_sequence(10, 2, 8) # {10, 20, 40, ..., 1280}

    # Arrays for European Put Option Pricing
    EP_ABT = np.empty(len(N))
    EP_ABT_approximation_error = np.empty(len(N))
    EP_ABT_linear_approximation_error = np.empty(len(N))
    EP_ABT_quadratic_approximation_error = np.empty(len(N))

    EP_ABT_delta = np.empty(len(N))
    EP_ABT_delta_error = np.empty(len(N))
    EP_ABT_gamma = np.empty(len(N))
    EP_ABT_gamma_error = np.empty(len(N))
    EP_ABT_theta = np.empty(len(N))
    EP_ABT_theta_error = np.empty(len(N))

    for i in range(len(N)):
        EP_ABT[i],EP_ABT_approximation_error[i],EP_ABT_linear_approximation_error[i], EP_ABT_quadratic_approximation_error[i], EP_ABT_delta[i], EP_ABT_delta_error[i], EP_ABT_gamma[i], EP_ABT_gamma_error[i],EP_ABT_theta[i],EP_ABT_theta_error[i] = simulation(
            S0, K, T, r, q, sigma, N[i], 
            CallPut =  CallPut, 
            EuroAmer = EuroAmer, 
            method_name = method_name,
            V_exact = V_exact,
            Delta_exact = Delta_exact,
            Gamma_exact = Gamma_exact,
            Theta_exact = Theta_exact)
        

    binomial_tree_res = pd.DataFrame({
        "N": N,
        "V(N)": EP_ABT,
        "|V(N) - V_exact|": EP_ABT_approximation_error, 
        "N|V(N) - V_exact|": EP_ABT_linear_approximation_error,
        "N^2|V(N) - V_exact|": EP_ABT_quadratic_approximation_error,
        "Delta_1": EP_ABT_delta,
        "|Delta_1 - Delta_BS|": EP_ABT_delta_error,
        "Gamma_1": EP_ABT_gamma, 
        "|Gamma_1 - Gamma_BS|":EP_ABT_gamma_error,
        "Theta_1": EP_ABT_theta,
        "|Theta_1 - Theta_BS|": EP_ABT_theta_error,
    })

    return binomial_tree_res



### Binomial Tree

In [7]:
def binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put
    ## Flag 2 --> 0 for European, 1 for American
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = N//2
    if not N % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N+1)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        payoffs(V, i, S_temp1, K, flag1, flag2)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N+1):
        payoffs(V, i, S_temp2, K, flag1, flag2)
        S_temp2 *= u_bar
    
    ## Work backwards through the tree.
    for j in range(N - 1, -1, -1):
        if flag2:
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar
    return V[0]

def binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    """
    This is a modication of Kevin's binomial pricer
    In order to get V_1_0, V_1_1, V_2_0, V_2_1, V_2_2, ...
    Args:
        early_stop: the nth step from V_0
    """
    ## Flag 1 --> 0 for call, 1 for put
    ## Flag 2 --> 0 for European, 1 for American
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = N//2
    if not N % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N+1)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        payoffs(V, i, S_temp1, K, flag1, flag2)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N+1):
        payoffs(V, i, S_temp2, K, flag1, flag2)
        S_temp2 *= u_bar
    
    ## Work backwards through the tree.
    for j in range(N - 1, -1+early_stop, -1):
        if flag2:
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar
    return V[:(early_stop+1)]

In [8]:
binomial_tree_res = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                              CallPut =  CallPutType.Put , 
                              EuroAmer =EuroAmerType.EuropeanOption, 
                              method_name = 'binomial',
                              V_exact = V_BS,
                              Delta_exact = Delta_BS,
                              Gamma_exact = Gamma_BS,
                              Theta_exact = Theta_BS)
binomial_tree_res

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.703631,0.124202,1.242025,12.420250,0.306502,0.610156,0.011641,0.010559,-2.227566,0.090616
1,20,3.642832,0.063404,1.268083,25.361669,0.304525,0.608179,0.011277,0.010923,-2.178015,0.041065
2,40,3.582311,0.002882,0.115300,4.611987,0.303623,0.607277,0.011238,0.010962,-2.172174,0.035224
3,80,3.582365,0.002936,0.234918,18.793401,0.303807,0.607461,0.011089,0.011111,-2.154212,0.017262
4,160,3.585679,0.006251,1.000199,160.031855,0.303748,0.607402,0.011000,0.011199,-2.142895,0.005945
5,320,3.583016,0.003588,1.148124,367.399785,0.303721,0.607375,0.010975,0.011225,-2.139740,0.002790
6,640,3.581148,0.001719,1.100460,704.294342,0.303686,0.607339,0.010965,0.011235,-2.138379,0.001429
7,1280,3.579952,0.000524,0.671047,858.940688,0.303663,0.607317,0.010960,0.011240,-2.137825,0.000876


In [13]:
binomial_tree_res.to_csv('binomial_tree_res.csv',index=False)

### Average Binomial Tree

In [9]:
def avg_binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (binomial_pricer(S0, K, T, r, q, sigma, N+1, flag1, flag2) + binomial_pricer(S0, K, T, r, q, sigma, N, flag1, flag2))/2 ## Simple average.


def avg_binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (
        binomial_pricer_early_stop(S0, K, T, r, q, sigma, N+1, flag1, flag2, early_stop) 
        + binomial_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop)
    )/2 ## Simple average.

In [10]:
avg_binomial_tree_res = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                              CallPut =  CallPutType.Put , 
                              EuroAmer =EuroAmerType.EuropeanOption, 
                              method_name = 'avg_binomial',
                              V_exact = V_BS,
                              Delta_exact = Delta_BS,
                              Gamma_exact = Gamma_BS,
                              Theta_exact = Theta_BS)
avg_binomial_tree_res

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.608162,0.028734,0.287340,2.873402,0.297369,0.601022,0.011342,0.010858,-2.167177,0.030227
1,20,3.602012,0.022584,0.451672,9.033434,0.300565,0.604219,0.011106,0.011094,-2.146797,0.009847
2,40,3.593785,0.014357,0.574284,22.971361,0.302062,0.605716,0.011016,0.011184,-2.139979,0.003029
3,80,3.586726,0.007298,0.583830,46.706402,0.302869,0.606523,0.010984,0.011216,-2.138349,0.001399
4,160,3.583241,0.003813,0.610034,97.605441,0.303261,0.606915,0.010968,0.011232,-2.137526,0.000576
5,320,3.580897,0.001469,0.470149,150.447705,0.303449,0.607103,0.010962,0.011238,-2.137445,0.000495
6,640,3.580235,0.000807,0.516329,330.450291,0.303553,0.607206,0.010958,0.011242,-2.137160,0.000210
7,1280,3.579939,0.000511,0.654415,837.651463,0.303605,0.607258,0.010955,0.011245,-2.137001,0.000051


In [14]:
avg_binomial_tree_res.to_csv('avg_binomial_tree_res.csv',index=False)

### Binomial Black–Scholes

In [11]:
def BBS_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = (N-1)//2
    if not (N-1) % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        if not flag1: ## Call option.
            V[i] = bs_call(S_temp1, K, dt, r, q, sigma)
        else: ## Put option.
            V[i] = bs_put(S_temp1, K, dt, r, q, sigma)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N):
        if not flag1:
            V[i] = bs_call(S_temp2, K, dt, r, q, sigma)
        else:
            V[i] = bs_put(S_temp2, K, dt, r, q, sigma)
        S_temp2 *= u_bar
            
    ## Work backwards through the tree.
    for j in range(N - 2, -1, -1):
        if flag2: ## American option.
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar

    return V[0]


def BBS_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.
    
    ## Define parameters.
    dt = T/N ## Time step.
    u = np.exp(sigma*np.sqrt(dt))
    d = 1/u
    d_bar = d**2
    u_bar = u**2
    q_up = (np.exp((r-q)*dt) - d) / (u - d)
    q_down = 1 - q_up
    disc = np.exp(-r*dt)
    discp = disc*q_up
    disc1p = disc*q_down
    mid = (N-1)//2
    if not (N-1) % 2:
        S_temp1 = S0
    else:
        S_temp1 = S0*d
    S_temp2 = S_temp1*u_bar
    V = np.empty(N)

    ## Calculate terminal payoffs.
    for i in range(mid,-1,-1):
        if not flag1: ## Call option.
            V[i] = bs_call(S_temp1, K, dt, r, q, sigma)
        else: ## Put option.
            V[i] = bs_put(S_temp1, K, dt, r, q, sigma)
        S_temp1 *= d_bar
            
    for i in range(mid+1,N):
        if not flag1:
            V[i] = bs_call(S_temp2, K, dt, r, q, sigma)
        else:
            V[i] = bs_put(S_temp2, K, dt, r, q, sigma)
        S_temp2 *= u_bar
            
    ## Work backwards through the tree.
    for j in range(N - 2, -1+early_stop, -1):
        if flag2: ## American option.
            S_temp = S0 * d**j
        for k in range(j + 1):
            V[k] = discp * V[k+1] + disc1p * V[k]
            
            ## American Option Adjustment.
            if flag2: 
                if not flag1: ## American call.
                    V[k] = max(V[k], S_temp - K)
                else: ## American put.
                    V[k] = max(V[k], K - S_temp)
                S_temp = S_temp * u_bar

    return V[:(early_stop+1)]


In [15]:
BBS_binomial_tree_res = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                              CallPut =  CallPutType.Put , 
                              EuroAmer =EuroAmerType.EuropeanOption, 
                              method_name = 'BBS',
                              V_exact = V_BS,
                              Delta_exact = Delta_BS,
                              Gamma_exact = Gamma_BS,
                              Theta_exact = Theta_BS)
BBS_binomial_tree_res

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.617453,0.038025,0.380251,3.802509,0.304397,0.608050,0.012106,0.010094,-2.281462,0.144512
1,20,3.599228,0.019800,0.396005,7.920103,0.304118,0.607771,0.011488,0.010712,-2.204251,0.067301
2,40,3.589882,0.010454,0.418150,16.725992,0.303903,0.607557,0.011209,0.010991,-2.169205,0.032255
3,80,3.584714,0.005286,0.422893,33.831461,0.303774,0.607428,0.011079,0.011121,-2.152787,0.015837
4,160,3.582031,0.002603,0.416491,66.638588,0.303716,0.607370,0.011016,0.011184,-2.144844,0.007895
5,320,3.580729,0.001301,0.416413,133.252032,0.303684,0.607338,0.010985,0.011215,-2.140880,0.003930
6,640,3.580081,0.000652,0.417545,267.228599,0.303669,0.607323,0.010969,0.011231,-2.138911,0.001961
7,1280,3.579758,0.000330,0.422156,540.360008,0.303662,0.607315,0.010961,0.011239,-2.137928,0.000978


In [16]:
BBS_binomial_tree_res.to_csv('BBS_binomial_tree_res.csv',index=False)

### Binomial Black–Scholes with Richardson Extrapolation

In [17]:
def BBSN_pricer(S0, K, T, r, q, sigma, N, flag1, flag2):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.
    return (2 * BBS_pricer(S0, K, T, r, q, sigma, N, flag1, flag2) - BBS_pricer(S0, K, T, r, q, sigma, N//2+1, flag1, flag2))

def BBSN_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop):
    ## Flag 1 --> 0 for call, 1 for put.
    ## Flag 2 --> 0 for European, 1 for American.

    return (
        2*BBS_pricer_early_stop(S0, K, T, r, q, sigma, N, flag1, flag2, early_stop) 
        - BBS_pricer_early_stop(S0, K, T, r, q, sigma, N//2+1, flag1, flag2, early_stop)
    )



In [18]:
BBSN_binomial_tree_res = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                              CallPut =  CallPutType.Put , 
                              EuroAmer =EuroAmerType.EuropeanOption, 
                              method_name = 'BBSN',
                              V_exact = V_BS,
                            Delta_exact = Delta_BS,
                            Gamma_exact = Gamma_BS,
                            Theta_exact = Theta_BS)
BBSN_binomial_tree_res

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.594108,0.014680,0.146803,1.468035,0.215328,0.518981,0.000287,0.021913,-0.560134,1.576816
1,20,3.582237,0.002809,0.056186,1.123721,0.197398,0.501052,-0.001439,0.023639,-0.291168,1.845782
2,40,3.580478,0.001050,0.041992,1.679682,0.187991,0.491645,-0.002356,0.024555,-0.148205,1.988745
3,80,3.579988,0.000559,0.044756,3.580466,0.183001,0.486655,-0.002824,0.025024,-0.074436,2.062514
4,160,3.579513,0.000085,0.013650,2.183941,0.180451,0.484105,-0.003057,0.025257,-0.037500,2.099450
5,320,3.579393,0.000035,0.011283,3.610707,0.179169,0.482822,-0.003175,0.025375,-0.018884,2.118066
6,640,3.579402,0.000026,0.016694,10.684464,0.178523,0.482176,-0.003234,0.025434,-0.009456,2.127494
7,1280,3.579421,0.000007,0.008547,10.940227,0.178200,0.481854,-0.003264,0.025464,-0.004728,2.132222


In [19]:
BBSN_binomial_tree_res.to_csv('BBSN_binomial_tree_res.csv',index=False)

## Binomial Tree Methods for American Options

In [20]:
''' 1. Compute the value of an American Put with the same parameters by using an average binomial
tree with 10, 000 and 10, 001 time steps and denote it by Vexact
'''

N = 10000
S0 = 54
K = 50
T = 1
r = 0.0375
q = 0.01
sigma = 0.29

V_exact,redunadant_1, redunadant_2, redunadant_3,Delta_exact,redunadant_5, Gamma_exact, redunadant_7, Theta_exact, redunadant_9= simulation(S0, K, T, r, q, sigma, N, 
            CallPut =  CallPutType.Put, 
            EuroAmer = EuroAmerType.AmericanOption, 
            method_name = 'binomial',
            V_exact = V_BS,
            Delta_exact = Delta_BS,
            Gamma_exact = Gamma_BS,
            Theta_exact = Theta_BS
            )


black_scholes_res = pd.DataFrame({
    "V_exact": V_exact,
    "Delta_exact": Delta_exact,
    "Gamma_exact": Gamma_exact,
    "Theta_exact": Theta_exact,
}, index=[0])
print(black_scholes_res)

    V_exact  Delta_exact  Gamma_exact  Theta_exact
0  3.668458     0.314152     0.011797      -2.2691


2. Price the American put option

### Binomial Tree


In [21]:
binomial_tree_res_am = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                            CallPut =  CallPutType.Put, 
                            EuroAmer = EuroAmerType.AmericanOption, 
                            method_name = 'binomial',
                            V_exact = V_exact,
                            Delta_exact = Delta_exact,
                            Gamma_exact = Gamma_exact,
                            Theta_exact = Theta_exact)
binomial_tree_res_am

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.787176,0.118717,1.187174,11.871743,0.315970,0.001818,0.012405,0.000608,-2.347192,0.078092
1,20,3.735565,0.067107,1.342131,26.842622,0.314801,0.000650,0.012065,0.000269,-2.302682,0.033582
2,40,3.678748,0.010290,0.411600,16.464006,0.314303,0.000151,0.012047,0.000251,-2.300507,0.031407
3,80,3.671770,0.003311,0.264919,21.193493,0.314276,0.000125,0.011927,0.000130,-2.285620,0.016520
4,160,3.675563,0.007105,1.136772,181.883550,0.314225,0.000073,0.011835,0.000038,-2.273875,0.004775
5,320,3.672046,0.003587,1.147993,367.357868,0.314194,0.000042,0.011815,0.000018,-2.271383,0.002283
6,640,3.670161,0.001703,1.089843,697.499572,0.314172,0.000020,0.011806,0.000009,-2.270224,0.001124
7,1280,3.669049,0.000591,0.756205,967.942178,0.314159,0.000008,0.011802,0.000005,-2.269749,0.000649


In [26]:
binomial_tree_res_am.to_csv('binomial_tree_res_am.csv',index=False)

### Average binomial Tree

In [22]:
arg_binomial_tree_res_am = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                            CallPut =  CallPutType.Put, 
                            EuroAmer =EuroAmerType.AmericanOption, 
                            method_name = 'avg_binomial',
                            V_exact = V_exact,
                            Delta_exact = Delta_exact,
                            Gamma_exact = Gamma_exact,
                            Theta_exact = Theta_exact)
arg_binomial_tree_res_am

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.711366,0.042908,0.429077,4.290767,0.307803,0.006349,0.012057,2.604521e-04,-2.283066,0.013966
1,20,3.693380,0.024921,0.498423,9.968458,0.310840,0.003312,0.011903,1.063171e-04,-2.272588,0.003488
2,40,3.686147,0.017689,0.707555,28.302187,0.312480,0.001672,0.011823,2.658551e-05,-2.267353,0.001747
3,80,3.676871,0.008412,0.672973,53.837800,0.313307,0.000844,0.011810,1.367320e-05,-2.268259,0.000841
4,160,3.672898,0.004439,0.710265,113.642411,0.313729,0.000422,0.011802,4.826615e-06,-2.268421,0.000679
5,320,3.670417,0.001959,0.626740,200.556785,0.313938,0.000214,0.011800,3.095841e-06,-2.268845,0.000255
6,640,3.669461,0.001003,0.641819,410.764303,0.314044,0.000107,0.011798,8.446870e-07,-2.268884,0.000217
7,1280,3.669000,0.000542,0.693384,887.531871,0.314098,0.000054,0.011796,3.599903e-07,-2.268893,0.000207


In [27]:
arg_binomial_tree_res_am.to_csv('arg_binomial_tree_res_am.csv',index=False)

### Binomial Black–Scholes

In [23]:
BBS_binomial_tree_res_am = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                            CallPut =  CallPutType.Put, 
                            EuroAmer = EuroAmerType.AmericanOption, 
                            method_name = 'BBS',
                            V_exact = V_exact,
                            Delta_exact = Delta_exact,
                            Gamma_exact = Gamma_exact,
                            Theta_exact = Theta_exact)
BBS_binomial_tree_res_am

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.689851,0.021393,0.213930,2.139301,0.313807,0.000345,0.012999,0.001202,-2.417125,0.148025
1,20,3.686245,0.017786,0.355723,7.114465,0.314400,0.000248,0.012330,0.000533,-2.335715,0.066615
2,40,3.678889,0.010431,0.417233,16.689329,0.314328,0.000177,0.012046,0.000249,-2.300398,0.031298
3,80,3.673757,0.005299,0.423886,33.910898,0.314242,0.000091,0.011919,0.000122,-2.284422,0.015322
4,160,3.671408,0.002950,0.471996,75.519347,0.314205,0.000054,0.011855,0.000058,-2.276455,0.007355
5,320,3.669977,0.001519,0.486072,155.542991,0.314179,0.000027,0.011825,0.000028,-2.272667,0.003567
6,640,3.669214,0.000755,0.483373,309.358968,0.314165,0.000014,0.011810,0.000014,-2.270810,0.001710
7,1280,3.668799,0.000340,0.435417,557.333534,0.314158,0.000006,0.011803,0.000006,-2.269904,0.000804


In [28]:
BBS_binomial_tree_res_am.to_csv('BBS_binomial_tree_res_am.csv',index=False)

### Binomial Black–Scholes with Richardson Extrapolation

In [24]:
BBSN_binomial_tree_res_am = approximate_binomial_tree_res(S0, K, T, r, q, sigma, 
                            CallPut =  CallPutType.Put, 
                            EuroAmer = EuroAmerType.AmericanOption, 
                            method_name = 'BBSN',
                            V_exact = V_exact,
                            Delta_exact = Delta_exact,
                            Gamma_exact = Gamma_exact,
                            Theta_exact = Theta_exact)
BBSN_binomial_tree_res_am

,N,V(N),|V(N) - V_exact|,N|V(N) - V_exact|,N^2|V(N) - V_exact|,Delta_1,|Delta_1 - Delta_BS|,Gamma_1,|Gamma_1 - Gamma_BS|,Theta_1,|Theta_1 - Theta_BS|
0,10,3.690401,0.021943,0.219427,2.194274,0.224503,0.089649,0.000330,0.011467,-0.588941,1.680159
1,20,3.673310,0.004851,0.097027,1.940540,0.204525,0.109626,-0.001404,0.013201,-0.312710,1.956390
2,40,3.673966,0.005508,0.220306,8.812240,0.194714,0.119438,-0.002421,0.014218,-0.155851,2.113249
3,80,3.669807,0.001349,0.107910,8.632798,0.189387,0.124765,-0.002895,0.014692,-0.080479,2.188622
4,160,3.668762,0.000304,0.048573,7.771752,0.186706,0.127445,-0.003143,0.014940,-0.041302,2.227798
5,320,3.668611,0.000152,0.048749,15.599675,0.185371,0.128781,-0.003275,0.015072,-0.020743,2.248357
6,640,3.668483,0.000025,0.015929,10.194336,0.184697,0.129454,-0.003341,0.015138,-0.010433,2.258667
7,1280,3.668400,0.000058,0.074345,95.161093,0.184361,0.129790,-0.003374,0.015171,-0.005259,2.263841


In [29]:
BBSN_binomial_tree_res_am.to_csv('BBSN_binomial_tree_res_am.csv',index=False)

## Variance Reduction Method

In [25]:
## output module of Binomial Tree Error Approximate
def generate_geometric_sequence(start, ratio, length):
    """
    e.g. start = 10, ratio = 2, length = 5
    return [10, 20, 40, 80, 160]
    """
    progression = np.empty(length, dtype=int)
    for i in range(0, length):
        curr_term = start * pow(ratio, i)
        progression[i] = int(curr_term)
    return progression
    
def simulation_VR(S0, K, T, r, q, sigma, N, CallPut, EuroAmer, method_name,V_exact,Delta_exact,Gamma_exact,Theta_exact):
        # preparation
        f1 = eval(f'{method_name}_pricer_early_stop')
        f2 = eval(f'{method_name}_pricer')
        V_1_0, V_1_1 = f1(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer,
            early_stop=1
        )
        V_2_0, V_2_1, V_2_2 = f1(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer,
            early_stop=2
        )
        V_0_0 = f2(
            S0, K, T, r, q, sigma, N, CallPut, EuroAmer
        )
        u = u_(T, sigma, N)
        d = d_(T, sigma, N)
        S_1_0 = S0 * u
        S_1_1 = S0 * d
        S_2_0 = S0 * u**2
        S_2_1 = S0 * u * d
        S_2_2 = S0 * d**2
        
        # value from binomial tree pricer
        EP_ABT = f2(S0, K, T, r, q, sigma, N, CallPut, EuroAmer)
        # approximation error
        EP_ABT_approximation_error = approximation_error(EP_ABT, V_exact)
        EP_ABT_linear_approximation_error = linear_approximation_error(EP_ABT, V_exact, N)
        EP_ABT_quadratic_approximation_error = quadratic_approximation_error(EP_ABT, V_exact, N)
        
        EP_ABT_delta = delta_1(V_1_0, V_1_1, S_1_0, S_1_1)
        EP_ABT_delta_error = approximation_error(EP_ABT_delta , Delta_exact)
        EP_ABT_gamma = gamma_1(V_2_0, V_2_1, V_2_2, S_2_0, S_2_1, S_2_2)
        EP_ABT_gamma_error = approximation_error(EP_ABT_gamma, Gamma_exact)
        EP_ABT_theta = theta_1(V_2_1, V_0_0, T, N)
        EP_ABT_theta_error = approximation_error(EP_ABT_theta, Theta_exact)

        return EP_ABT,EP_ABT_approximation_error,EP_ABT_linear_approximation_error, EP_ABT_quadratic_approximation_error, EP_ABT_delta, EP_ABT_delta_error, EP_ABT_gamma, EP_ABT_gamma_error,EP_ABT_theta,EP_ABT_theta_error

        
def approximate_binomial_tree_res_VR(S0, K, T, r, q, sigma, CallPut, EuroAmer, method_name,V_exact,Delta_exact,Gamma_exact,Theta_exact):
    # Average Binomial Tree
    # number of steps to use 
    N = generate_geometric_sequence(10, 2, 8) # {10, 20, 40, ..., 1280}

    # Arrays for European Put Option Pricing
    EP_ABT = np.empty(len(N))
    EP_ABT_approximation_error = np.empty(len(N))
    EP_ABT_linear_approximation_error = np.empty(len(N))
    EP_ABT_quadratic_approximation_error = np.empty(len(N))

    EP_ABT_delta = np.empty(len(N))
    EP_ABT_delta_error = np.empty(len(N))
    EP_ABT_gamma = np.empty(len(N))
    EP_ABT_gamma_error = np.empty(len(N))
    EP_ABT_theta = np.empty(len(N))
    EP_ABT_theta_error = np.empty(len(N))

    for i in range(len(N)):
        EP_ABT[i],EP_ABT_approximation_error[i],EP_ABT_linear_approximation_error[i], EP_ABT_quadratic_approximation_error[i], EP_ABT_delta[i], EP_ABT_delta_error[i], EP_ABT_gamma[i], EP_ABT_gamma_error[i],EP_ABT_theta[i],EP_ABT_theta_error[i] = simulation_VR(
            S0, K, T, r, q, sigma, N[i], 
            CallPut =  CallPut, 
            EuroAmer = EuroAmer, 
            method_name = method_name,
            V_exact = V_exact,
            Delta_exact = Delta_exact,
            Gamma_exact = Gamma_exact,
            Theta_exact = Theta_exact)
        

    binomial_tree_res = pd.DataFrame({
        "N": N,
        "V(N)": EP_ABT,
        "|V(N) - V_exact|": EP_ABT_approximation_error, 
        "N|V(N) - V_exact|": EP_ABT_linear_approximation_error,
        "N^2|V(N) - V_exact|": EP_ABT_quadratic_approximation_error,
        "Delta_1": EP_ABT_delta,
        "|Delta_1 - Delta_BS|": EP_ABT_delta_error,
        "Gamma_1": EP_ABT_gamma, 
        "|Gamma_1 - Gamma_BS|":EP_ABT_gamma_error,
        "Theta_1": EP_ABT_theta,
        "|Theta_1 - Theta_BS|": EP_ABT_theta_error,
    })

    return binomial_tree_res




